In [1]:
# import Glop linear solver package
from ortools.linear_solver import pywraplp as glp

In [2]:
# model parameters
project = ['Limited Warehouse Expansion', 'Extensive Warehouse Expansion', 'Test Market New Product', 'Advertising Campaign', 
    'Basic Research', 'Purchase New Equipment']
year = ['Year1','Year2','Year3']
npv = [4, 6, 10.5, 4, 8, 3]  # net present value of each project in thousand $
budget = [10.5, 7, 8.75]  # capital budget available each year in thousand $
cap_req = [[3, 2.5, 6, 2, 5, 1],  # capital requirement for each project in Year 1
           [1, 3.5, 4, 1.5, 1, 0.5],  # Year 2
           [4, 3.5, 5, 1.8, 4, 0.9]]  # Year 3

In [3]:
# initialize model object
mymodel = glp.Solver('Capital Budget', glp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)




In [4]:
# create decision variables
select = list(range(len(project)))
for j in range(len(project)):
    select[j] = mymodel.IntVar(0,1, project[j])
    
print('select = ', select)

select =  [Limited Warehouse Expansion, Extensive Warehouse Expansion, Test Market New Product, Advertising Campaign, Basic Research, Purchase New Equipment]


In [5]:
# create objective function
tot_npv = mymodel.Objective()
tot_npv.SetMaximization()
for j in range(len(project)):
    tot_npv.SetCoefficient(select[j], npv[j])

#display objective function
for j in range(len(project)):
    print(select[j], tot_npv.GetCoefficient(select[j]))

Limited Warehouse Expansion 4.0
Extensive Warehouse Expansion 6.0
Test Market New Product 10.5
Advertising Campaign 4.0
Basic Research 8.0
Purchase New Equipment 3.0


In [6]:
# create constraints
constr = list(range(len(year)))
for i in range(len(year)):
    constr[i] = mymodel.Constraint(0, budget[i])
    for j in range(len(project)):
        constr[i].SetCoefficient(select[j], cap_req[i][j])
    

In [7]:
# Solve the model and print optimal solution
status = mymodel.Solve()                 # solve mymodel and display the solution

print('Solution Status =', status)
print('Number of variables =', mymodel.NumVariables())
print('Number of constraints =', mymodel.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
print('Optimal Value = %.2f' % tot_npv.Value())

# Display optimal solution
for j in range(len(project)):
    if select[j].solution_value() == 1:
        print('%s %4.2f' % (project[j], select[j].solution_value()))

Solution Status = 0
Number of variables = 6
Number of constraints = 3
Optimal Solution:
Optimal Value = 17.50
Test Market New Product 1.00
Advertising Campaign 1.00
Purchase New Equipment 1.00


In [8]:
# display constraint information
print('Constraint \t LHS \t RHS \t Slack')
LHS = mymodel.ComputeConstraintActivities()
for i in range(len(year)):
    slack = budget[i] - LHS[i]
    print('%11s \t %3.1f \t %3.1f \t %4.1f' % (year[i], LHS[i], budget[i], slack))

Constraint 	 LHS 	 RHS 	 Slack
      Year1 	 9.0 	 10.5 	  1.5
      Year2 	 6.0 	 7.0 	  1.0
      Year3 	 7.7 	 8.8 	  1.0


In [10]:
# b. Alternative to the optimal example 
# store coefficients and RHS for cut constraint
# -select[0] - select[1] - select[2] + select[3] + select[4] <=2-1
# 0-1-0+1+1<=1
a = list()
M = 0

for i in range(len(select)):
    if select[i].solution_value() == 1:
        a.append(1)
        M+=1
    elif select[i].solution_value() == 0:
        a.append(-1)
print(M, a)

3 [-1, 1, -1, -1, 1, 1]


In [11]:
#add cut constraint
cut_constr = mymodel.Constraint(-mymodel.infinity(), M-1)
for i in range(len(project)):
    cut_constr.SetCoefficient(select[i], a[i])

    
#re-optimize and display next solution
mymodel.Solve()
print('Optimal Value =', tot_npv.Value())
for i in range(len(project)):
    if select[i].solution_value() == 1:
        print('\t %7s %6.2f' % (project[i], select[i].solution_value())) 
            
# relax cut constraint
cut_constr.SetBounds(-mymodel.infinity(),mymodel.infinity())

Optimal Value = 15.0
	 Advertising Campaign   1.00
	 Basic Research   1.00
	 Purchase New Equipment   1.00


In [ ]:
#Restart the kernel
#Run this separately from the 

In [9]:
#c. Cannot choose both Test Market and Advertising campsaign
# select[2] + selectp[3] <= 1
new_constr2 = mymodel.Constraint(0, 1)
new_constr2.SetCoefficient(select[2], 1)
new_constr2.SetCoefficient(select[3], 1)
mymodel.Solve()
print(tot_npv.Value())
# Display optimal solution
for j in range(len(project)):
    if select[j].solution_value() == 1:
        print('%s %4.2f' % (project[j], select[j].solution_value()))
    

17.0
Extensive Warehouse Expansion 1.00
Basic Research 1.00
Purchase New Equipment 1.00
